In [2]:
#Librerias

import numpy as np
import pandas as pd


In [11]:
#Importamos el dataset original para el EDA
df = pd.read_csv('movies_df.csv')

In [12]:
#Dejamos solo las columnas relevantes para nuestro modelo
df = df[['title', 'overview', 'vote_average']]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         45460 non-null  object 
 1   overview      44509 non-null  object 
 2   vote_average  45460 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.0+ MB


In [14]:
# Reemplazar los valores "ND" por NaN
df = df.replace('ND', np.nan)

# Eliminar las filas con valores NaN
df = df.dropna()

In [15]:
#Verificamos
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44506 entries, 0 to 45462
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         44506 non-null  object 
 1   overview      44506 non-null  object 
 2   vote_average  44506 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.4+ MB


In [16]:
#Quitamos los duplicados del dataset

df.drop_duplicates(subset=['title'], inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41371 entries, 0 to 45462
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         41371 non-null  object 
 1   overview      41371 non-null  object 
 2   vote_average  41371 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.3+ MB


In [17]:
# Obtenemos una muestra aleatoria del 20% de los datos para optimizar la funcion con el hardware que tendremos disponible una vez deployado
df = df.sample(frac=0.2, random_state=42)

print(df)


                            title  \
28393              The Time Being   
44788     Confidential Assignment   
3344                 Solar Crisis   
1828                 Going My Way   
16390           Countdown to Zero   
...                           ...   
26609                        Teza   
21223            What's in a Name   
33104        Naked Under the Moon   
40826            Matchmaker Santa   
37057  The Dark Side Of Chocolate   

                                                overview  vote_average  
28393  An artist meets a mysterious and wealthy benef...           4.9  
44788  When a crime organization from North Korea cro...           6.2  
3344   A huge solar flare is predicted to fry the Ear...           4.4  
1828   Youthful Father Chuck O'Malley (Bing Crosby) l...           6.9  
16390  A documentary about the escalating nuclear arm...           6.3  
...                                                  ...           ...  
26609  The Ethiopian intellectual Anberber r

In [18]:
# Reseteamos el indice para no tener problemas más adelante

df.reset_index(drop = True, inplace = True)

In [19]:
# De esta forma quedaria de momento
df.head(10)

,title,overview,vote_average
0,The Time Being,An artist meets a mysterious and wealthy benef...,4.9
1,Confidential Assignment,When a crime organization from North Korea cro...,6.2
2,Solar Crisis,A huge solar flare is predicted to fry the Ear...,4.4
3,Going My Way,Youthful Father Chuck O'Malley (Bing Crosby) l...,6.9
4,Countdown to Zero,A documentary about the escalating nuclear arm...,6.3
5,Street Trash,"When a liquor store owner finds a case of ""Ten...",5.6
6,If You Are the One,"Qin Fen, a funny, honest, single inventor, met...",6.5
7,Bloody Murder,Julie McConnell is one of a dozen camp counsel...,3.6
8,Comedy Central Roast of Larry the Cable Guy,Lisa Lampinelli hosts the Comedy Central Roast...,6.5
9,Baazi,Inspector Amar Damjee discovers a terrifying c...,7.5


In [20]:
# pasamos todas las letras a minúsculas en la columna "overview" para poder procesar su texto
df['overview'] = df['overview'].str.lower()

# Muestra los primeros 5 registros para verificar los cambios
print(df.head())


                     title                                           overview  \
0           The Time Being  an artist meets a mysterious and wealthy benef...   
1  Confidential Assignment  when a crime organization from north korea cro...   
2             Solar Crisis  a huge solar flare is predicted to fry the ear...   
3             Going My Way  youthful father chuck o'malley (bing crosby) l...   
4        Countdown to Zero  a documentary about the escalating nuclear arm...   

   vote_average  
0           4.9  
1           6.2  
2           4.4  
3           6.9  
4           6.3  


In [21]:
# Procesamiento del lenguaje con NLTK
#!pip install nltk
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\waiga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\waiga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\waiga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#Creamos una funcion para procesar el texto
def process_text(text):
    # Tokenización
    tokens = word_tokenize(text)
    
    # Conversión a minúsculas
    tokens = [token.lower() for token in tokens]
    
    # Eliminación de stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Unimos los tokens procesados en un solo texto
    processed_text = ' '.join(tokens)
    
    return processed_text

In [23]:
# Procesamos el texto
df['processed_overview'] = df['overview'].apply(process_text)

In [25]:
#Nuestro df ahora tendra una columna más con el texto procesado
df.head(10)

,title,overview,vote_average,processed_overview
0,The Time Being,an artist meets a mysterious and wealthy benef...,4.9,artist meet mysterious wealthy benefactor rela...
1,Confidential Assignment,when a crime organization from north korea cro...,6.2,crime organization north korea cross border en...
2,Solar Crisis,a huge solar flare is predicted to fry the ear...,4.4,huge solar flare predicted fry earth . astrona...
3,Going My Way,youthful father chuck o'malley (bing crosby) l...,6.9,youthful father chuck o'malley ( bing crosby )...
4,Countdown to Zero,a documentary about the escalating nuclear arm...,6.3,documentary escalating nuclear arm race .
5,Street Trash,"when a liquor store owner finds a case of ""ten...",5.6,liquor store owner find case `` tenafly viper ...
6,If You Are the One,"qin fen, a funny, honest, single inventor, met...",6.5,"qin fen , funny , honest , single inventor , m..."
7,Bloody Murder,julie mcconnell is one of a dozen camp counsel...,3.6,julie mcconnell one dozen camp counselor worki...
8,Comedy Central Roast of Larry the Cable Guy,lisa lampinelli hosts the comedy central roast...,6.5,lisa lampinelli host comedy central roast larr...
9,Baazi,inspector amar damjee discovers a terrifying c...,7.5,inspector amar damjee discovers terrifying cri...


In [ ]:
# Una vez procesado el texto quitamos la columna overview de nuestro dataset
df = df.drop('overview', axis=1)

# Exportamos a un .csv

df.to_csv('movies_ML_sample10.csv')